# Computing the Maximum Lyapunov Exponent via autograd

21.07.2025: Use this file as blueprint to update others

Genrating videos to better understand the dynamics via mFTLEs

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
import numpy as np
import importlib

# Juptyer magic: For export. Makes the plots size right for the screen 
%matplotlib inline
# %config InlineBackend.figure_format = 'retina'

%config InlineBackend.figure_formats = ['svg'] 


torch.backends.cudnn.deterministic = True
seed = np.random.randint(1,200)
# seed = 61 #59
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
print(seed)
g = torch.Generator()
g.manual_seed(seed)


# Data preparation

In [ ]:
cross_entropy = False #this leads to squared loss in the training
output_dim = 1 #for model architecture later, but need it already for dataloader
data_noise = 0.05
num_points = 5000
plotlim = [-3, 3]
subfolder = 'MLE_4params_less_noise' #all the files generated from this notebook get saved into this subfolder 

import os
if not os.path.exists(subfolder):
    os.makedirs(subfolder)


label = 'scalar' if (cross_entropy or output_dim == 1) else 'vector' #MSE allows both scalar and vector output, cross_entropy only allows "scalar" output here
print('label:', label)

from models.training import create_dataloader
dataloader, dataloader_viz = create_dataloader('moons', noise = data_noise, num_points = num_points, plotlim = plotlim, random_state = seed, label = label, filename = subfolder + '/trainingset')


## Model dynamics

In [ ]:
#Import of the model dynamics that describe the neural ODE
#The dynamics are based on the torchdiffeq package, that implements ODE solvers in the pytorch setting 
from models.neural_odes import NeuralODEvar

#for neural ODE based networks the network width is constant. In this example the input is 2 dimensional
hidden_dim, data_dim = 2, 2 
augment_dim = 0

#T is the end time of the neural ODE evolution, time_steps are the amount of discretization steps for the ODE solver
T, time_steps = 10, 100 #
step_size = T/time_steps
num_params = 5 #the number of distinct parameters present in the interval. they are spread equidistant over the interval [0, T]. As there are 100 time_steps, the interval is divided into 10 parts, each of length 1 with 10 time_steps per subinterval.
bound = 0.
turnpike = False
l2_factor = 0.1 #for the regularization term in the loss function, that penalizes large weights

non_linearity = 'relu' #'relu' #
architecture = 'inside' #outside




## Training and generating level sets

In [ ]:

num_epochs = 100 #number of optimization runs in which the dataset is used for gradient decent

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
anode = NeuralODEvar(device, data_dim, hidden_dim, output_dim=output_dim, augment_dim=augment_dim, non_linearity=non_linearity, 
                    architecture=architecture, T=T, time_steps=time_steps, num_params = num_params, cross_entropy=cross_entropy)


optimizer_anode = torch.optim.Adam(anode.parameters(), lr=1e-3) 

In [ ]:
from models.training import doublebackTrainer

trainer_anode = double_c__/adsfasdf_/fasdfdasfxorasdfasdfasdfbackTrainer(anode, optimizer_anode, device, cross_entropy=cross_entropy, turnpike = turnpike,
                         bound=bound, l2_factor=0.01, verbose = True) 
trainer_anode.train(dataloader, num_epochs)

In [ ]:
import plots.plots
importlib.reload(plots.plots)
from plots.plots import classification_levelsets

footnote = f'{num_epochs = }, {cross_entropy = }, {num_params = }, {l2_factor = },\n {time_steps = }, {output_dim = }, {data_noise = }, {seed = }'
        
fig_name_base = os.path.join(subfolder, 'levelsets')
classification_levelsets(anode, fig_name_base, footnote = footnote)
from IPython.display import Image
img1 = Image(filename = fig_name_base + '.png', width = 500)
display(img1)

plt.plot(trainer_anode.histories['epoch_loss_history'])
plt.xlim(0, len(trainer_anode.histories['epoch_loss_history']) - 1)
plt.ylim(0)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.show()

In [ ]:

import plots.gifs
importlib.reload(plots.gifs) # Reload the module to ensure the latest changes are applied
from plots.gifs import trajectory_gif
from IPython.display import Image

#the function trajectory_gif creates the evolution trajectories of the input data through the network
#the passing time corresponds to the depth of the network

for X_viz, y_viz in dataloader_viz:
    trajectory_gif(anode, X_viz[0:50], y_viz[0:50], timesteps=time_steps, filename = subfolder + '/trajectory', axlim = 8, dpi = 100)
    break

#Display of the generated gif
traj = Image(filename=subfolder + "/trajectory.gif")
display(traj)

## Lyapunov exponent computation

In [ ]:
from FTLE import LEs

#Example of how to use the LEs function to compute Lyapunov exponents
# Define inputs
input1 = torch.tensor([1, 0], dtype=torch.float32)
time_interval = torch.tensor([0, T], dtype=torch.float32)

les = LEs(input1, anode, time_interval=time_interval) #computes the Lyapunov exponents (l1>=l2>=...l_min) for the input1 over the time interval
print(les)


# Plot ANODE FTLEs

In [ ]:
from FTLE import LE_grid

output_max, output_min = LE_grid(anode,x_amount = 40)

In [ ]:
#testing grid plots
importlib.reload(plots.plots) # Reload the module to ensure the latest changes are applied
from plots.plots import plot_trajectory

ax_lim = 2
stepsize = T/time_steps
interval = torch.tensor([0., T], dtype=torch.float32)


anodeimg_max = plt.imshow(np.rot90(output_max), origin='upper', extent=(-ax_lim, ax_lim, -ax_lim, ax_lim),cmap = 'viridis')
vmin_max, vmax_max = anodeimg_max.get_clim()

plt.colorbar()  # Show color scale

plt.show()

In [ ]:
anodeimg_max = plt.imshow(np.rot90(output_max), origin='upper', extent=(-2, 2, -2, 2),cmap = 'viridis')
vmin_max, vmax_max = anodeimg_max.get_clim()

plt.colorbar()  # Show color scale
plt.savefig(subfolder + '/MLE_max.png',bbox_inches='tight', dpi=300, format='png', facecolor = 'white')
plt.close()

anodeimg_min = plt.imshow(np.rot90(output_min), origin='upper', extent=(-2, 2, -2, 2),cmap = 'viridis')#, norm=CenteredNorm(vcenter=0)) # cmap='viridis')
vmin_min, vmax_min = anodeimg_min.get_clim()

plt.colorbar()  # Show color scale
plt.savefig(subfolder + '/MLE_min.png',bbox_inches='tight', dpi=300, format='png', facecolor = 'white')
plt.close()

img1 = Image(filename = fig_name_base + '.png', width = 400)
img2 = Image(filename = subfolder + '/MLE_max.png', width = 400)
img3 = Image(filename = subfolder + '/MLE_min.png', width = 400)

display(img1,img2,img3)

## Video 1: FTLE for each subinterval

In [ ]:
importlib.reload(plots.plots)

import os

from plots.plots import plot_points_from_traj, input_to_traj_and_color, plot_vectorfield
from plots.plots import create_gif_subintervals
import imageio
import io

filename = subfolder + '/FTLE_per_param'
create_gif_subintervals(anode, le_density=40, filename=filename)
LEevo_test = Image(filename=filename + ".gif")
display(LEevo_test)

in the autonomous case any interval of same length should give the same LE. So the above gif makes sense!

In [ ]:
# def create_gif_shrinkingintervals(model, le_density = 30, point_density = 20, filename = 'LE_shrinking'):
#     """
#     Creates a GIF showing the evolution of input points and the future Lyapunov exponents. As the time progresses, the remaining Lyapunov exponent intervals shrink, reflecting the model's dynamics.
    
#     Parameters:
#     - model: The neural ODE model.
#     - le_amount: Number of Lyapunov exponent intervals.
#     - vf_amount: Number of vector field points.
#     - filename: Base name for the output files.
#     """
    
#     #density of FTLE grid


#     ###point plot preparation####
#     # Define the grid for vector field visualization
#     x = torch.linspace(-2,2,point_density)
#     y = torch.linspace(-2,2,point_density)
#     X, Y = torch.meshgrid(x, y)
#     inputs_grid = torch.stack([X,Y], dim=-1)
#     inputs_grid = inputs_grid.view(-1,2) #to be able to input all the grid values into the model at once

#     #compute traj and colors of grid inputs first, later no more evaluations of model needed
#     trajs, colors = input_to_traj_and_color(model, inputs_grid)

    
#     T = model.T
#     step_size = T/model.time_steps #time step for the integration
#     eps = 0.1 * step_size #this should make sure we stay inside an interval with constant parameters for the integration
#     t_values = np.arange(0 + eps, T-step_size, step_size) #all discretization steps computed in the nODE flow
    
#     images = []
    
#     for t in t_values:
    
#         ###FTLE plot
#         le_interval = torch.tensor([t, T], dtype=torch.float32)

#         output_max, _ = LE_grid(model, le_density, le_interval)
#         plt.imshow(np.rot90(output_max), origin='upper', extent=(-2, 2, -2, 2), cmap='viridis')
#         cbar = plt.colorbar()
#         cbar.ax.tick_params(labelsize=10)  # Adjust tick label size
        
#         ### Plot points from trajectory
#         plot_points_from_traj(trajs, colors, model, t) #this + step_size here does not make sense yet must still be explained.
        
#         plt.gca().set_aspect('equal', adjustable='box')  # more robust than plt.axis('equal')
#         plt.xlim(-2, 2)
#         plt.ylim(-2, 2)
#         plt.xlabel(r"$x_1$", fontsize=5)
#         plt.ylabel(r"$x_2$", fontsize=5)
#         plt.tick_params(axis='both', which='major', labelsize=5)
#         plt.title(f'Time {t:.2f}, FTLE interval [{le_interval[0].item():.2f}, {le_interval[1].item():.2f}] ', fontsize = 7)
        
#         buf = io.BytesIO()
#         plt.savefig(buf, bbox_inches='tight', dpi=200, format='png', facecolor='white')
#         buf.seek(0)  # rewind to beginning of buffer
#         images.append(imageio.imread(buf))  # or use PIL.Image.open(buf)
#         buf.close()
#         plt.close()
#         print(f'Saved plot for t={t:.1f}')
    
#     imageio.mimsave(filename + '.gif', images, fps=2)
    


## Video 2: FTLE for shrinking interval

In [ ]:
importlib.reload(plots.plots) # Reload the module to ensure the latest changes are applied
from plots.plots import create_gif_shrinkingintervals


create_gif_shrinkingintervals(anode, filename=subfolder + '/shrinking_LE')
LEevo_test = Image(filename=subfolder + '/shrinking_LE.gif')
display(LEevo_test)